## Statistics Assignment

All of the following are based on data from your project. Each student has to perform analysis on their own no collaboration between team members. Here are the datasets to be analyzed by team datasets:

***
### ***For Part 1 and 2***

##### Library-Computer-Usage-Analysis
* Computer Utilization Data by Date-Time

##### Volag
* Flight Delay Data by Date-Time

##### Slipper-Streets
* Crash Data by Date-Time

##### Corpus
* Reviews of Electronic product (laptop) by Date-Time

##### SteamConnect
* Early Access Score by Release Date-Time

##### Toxic-Crusaders
* Chemical Industry Release (pick a particular industry) by Date-Time

##### Uni-X
* Repayment Rate for Female gender by Date-Time

##### WRF
* Migration count by Date-Time


### Part 1

* **Conduct Decriptive Analytics (Mean, Median, Quartile) calculation by each division of Date-Time (most probably year or 6 month duration, if you have shorter use 1 month)**
* **Calculate divergence of mean and median in your data**
* **Visualize the data and draw inferences**

### Part 2
* **Conduct Probablity distibution analysis based on the data. Analyze your data based on the type of distribution it best fits (for PDF and CDF)**
* **Conduct Method of Moments analysis on your data to suggest the best fit distribution. Visualize the results**

***
***

### ***For Part 3***
* Compare with the variable with other variables in your project

### Part 3
* **Formulate a null hypothesis and evaluate it, perform correlation measures, and construct a linear regression model**

***

In [53]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
import random

pd.options.display.max_rows = 999

## Do some more data wrangling

In [54]:
path = os.path.join('..','Data','flights_weather_1.csv')

### Our dataset is a little bit large

Since our dataset grew to about 2GB in size after merging it with the weather data, it's a bit much to load into
the memory of my system and also process. For that reason, a random sample of the dataset should be taken. 

In [55]:
#lines = sum(1 for l in open(path))

with open(path) as f:
    lines = sum(1 for l in f)
    
# sample ~30% of the dataset
sample_size = int(lines / 30)
skip = random.sample(range(1,lines), lines - sample_size)
# use skip lines
sample = pd.read_csv(path, skiprows=skip, dtype='str')

In [56]:
sample = sample.loc[:, ~sample.columns.str.contains('^Unnamed')]
sample = sample.drop(['YEAR','MONTH','DAY','OR_MAX','OR_MIN','DES_MAX','DES_MIN'],axis=1)

In [57]:
sample.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
0,6,EV,2509,N902EV,DFW,GGG,1010,1005.0,-5.0,19.0,...,24.0,10.0,24.0,5.0,24.0,11.1,19.0,0.00G,999.9,0
1,6,MQ,3392,N682MQ,CLE,ORD,1010,1028.0,18.0,11.0,...,24.0,10.0,24.0,5.9,24.0,12.0,999.9,0.01G,1.2,0
2,6,US,621,N652AW,PHX,SNA,1020,1017.0,-3.0,28.0,...,24.0,8.6,24.0,4.7,24.0,9.9,999.9,0.00G,999.9,0
3,6,US,623,N195UW,PHX,SAN,1020,1014.0,-6.0,36.0,...,24.0,6.2,24.0,4.3,24.0,9.9,999.9,0.00G,999.9,0
4,6,WN,2455,N387SW,STL,TPA,1020,1023.0,3.0,9.0,...,24.0,10.0,24.0,8.8,24.0,22.0,999.9,0.27G,999.9,0


We need to replace some values in our sample. A value of 999.9~ means that there was not a recorded value for that particular day. So, for precipitation we will replace non-recordings with 0, and for the other values such as wind speed and temperature it makes more sense to replace them with the mean value of the values which aren't 999.9~

In [58]:
print(type(sample['OR_PRCP'][0]))
flags = ['A','B','C','D','E','F', 'G', 'H', 'I']
sample['OR_PRCP'] = sample['OR_PRCP'].apply(lambda x: str(x)[:-1] if str(x)[-1] in flags else str(x))
sample['DES_PRCP'] = sample['DES_PRCP'].apply(lambda x: str(x)[:-1] if str(x)[-1] in flags else str(x))

<class 'str'>


In [59]:
sample.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
0,6,EV,2509,N902EV,DFW,GGG,1010,1005.0,-5.0,19.0,...,24.0,10.0,24.0,5.0,24.0,11.1,19.0,0.00,999.9,0
1,6,MQ,3392,N682MQ,CLE,ORD,1010,1028.0,18.0,11.0,...,24.0,10.0,24.0,5.9,24.0,12.0,999.9,0.01,1.2,0
2,6,US,621,N652AW,PHX,SNA,1020,1017.0,-3.0,28.0,...,24.0,8.6,24.0,4.7,24.0,9.9,999.9,0.00,999.9,0
3,6,US,623,N195UW,PHX,SAN,1020,1014.0,-6.0,36.0,...,24.0,6.2,24.0,4.3,24.0,9.9,999.9,0.00,999.9,0
4,6,WN,2455,N387SW,STL,TPA,1020,1023.0,3.0,9.0,...,24.0,10.0,24.0,8.8,24.0,22.0,999.9,0.27,999.9,0


We write the sample to a new file after creating it and cleaning it up a bit.

In [60]:
sample.to_csv(os.path.join('..','Data','fw_sample.csv'))

In [61]:
dtypes = {
    'OR_PRCP': 'float64',  
    'DES_PRCP': 'float64',
}

fw_samp = pd.read_csv(os.path.join('..','Data','fw_sample.csv'), low_memory=False)

fw_samp.head()

,Unnamed: 0,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
0,0,6,EV,2509,N902EV,DFW,GGG,1010,1005.0,-5.0,...,24.0,10.0,24.0,5.0,24.0,11.1,19.0,0.00,999.9,0.0
1,1,6,MQ,3392,N682MQ,CLE,ORD,1010,1028.0,18.0,...,24.0,10.0,24.0,5.9,24.0,12.0,999.9,0.01,1.2,0.0
2,2,6,US,621,N652AW,PHX,SNA,1020,1017.0,-3.0,...,24.0,8.6,24.0,4.7,24.0,9.9,999.9,0.00,999.9,0.0
3,3,6,US,623,N195UW,PHX,SAN,1020,1014.0,-6.0,...,24.0,6.2,24.0,4.3,24.0,9.9,999.9,0.00,999.9,0.0
4,4,6,WN,2455,N387SW,STL,TPA,1020,1023.0,3.0,...,24.0,10.0,24.0,8.8,24.0,22.0,999.9,0.27,999.9,0.0


In [62]:
fw_samp = fw_samp.loc[:, ~fw_samp.columns.str.contains('^Unnamed')]

# for the sake of simplicity, because our data set is huge and still pretty messy, just replace
# all 999.9 or 9999.9 values with 0
for col in fw_samp.columns:
    fw_samp[col] = fw_samp[col].replace(999.9,0)
    fw_samp[col] = fw_samp[col].replace(9999.9,0)
    fw_samp[col] = fw_samp[col].replace('999.9','0')
    fw_samp[col] = fw_samp[col].replace('9999.9','0')
       
fw_samp.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
0,6,EV,2509,N902EV,DFW,GGG,1010,1005.0,-5.0,19.0,...,24.0,10.0,24.0,5.0,24.0,11.1,19.0,0.00,0.0,0.0
1,6,MQ,3392,N682MQ,CLE,ORD,1010,1028.0,18.0,11.0,...,24.0,10.0,24.0,5.9,24.0,12.0,0.0,0.01,1.2,0.0
2,6,US,621,N652AW,PHX,SNA,1020,1017.0,-3.0,28.0,...,24.0,8.6,24.0,4.7,24.0,9.9,0.0,0.00,0.0,0.0
3,6,US,623,N195UW,PHX,SAN,1020,1014.0,-6.0,36.0,...,24.0,6.2,24.0,4.3,24.0,9.9,0.0,0.00,0.0,0.0
4,6,WN,2455,N387SW,STL,TPA,1020,1023.0,3.0,9.0,...,24.0,10.0,24.0,8.8,24.0,22.0,0.0,0.27,0.0,0.0


In [63]:
fw_samp.describe()

,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
count,149166.000000,149166.000000,149166.000000,147626.000000,147626.000000,147567.000000,147567.000000,149166.000000,147157.000000,147157.000000,...,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000,133058.000000
mean,3.919787,2155.781928,1328.999551,1333.376417,8.880597,15.968414,1355.319265,142.004981,137.042506,113.666057,...,23.920178,9.425185,23.971013,6.982064,23.971396,14.038063,13.967744,0.240906,0.037241,10224.269537
std,2.000461,1747.467744,486.661356,499.195852,36.269435,8.604161,500.829650,75.425127,74.381679,72.420773,...,1.089442,1.222590,0.648418,3.081953,0.563682,4.988963,12.217380,3.470015,0.413274,25606.658551
min,1.000000,1.000000,3.000000,1.000000,-39.000000,1.000000,1.000000,21.000000,17.000000,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,725.000000,915.000000,918.000000,-5.000000,11.000000,933.000000,86.000000,82.000000,60.000000,...,24.000000,9.500000,24.000000,4.800000,24.000000,11.100000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,1681.000000,1325.000000,1329.000000,-2.000000,14.000000,1341.000000,123.000000,118.000000,94.000000,...,24.000000,10.000000,24.000000,6.600000,24.000000,13.000000,17.100000,0.000000,0.000000,0.000000
75%,6.000000,3173.000000,1731.000000,1740.000000,7.000000,19.000000,1755.000000,173.000000,168.000000,144.000000,...,24.000000,10.000000,24.000000,8.700000,24.000000,17.100000,22.900000,0.040000,0.000000,10000.000000
max,7.000000,7438.000000,2359.000000,2400.000000,1239.000000,154.000000,2400.000000,683.000000,690.000000,651.000000,...,24.000000,19.200000,24.000000,31.200000,24.000000,46.000000,69.900000,99.990000,22.800000,111110.000000


In [64]:
fw_samp.DATE = pd.to_datetime(fw_samp.DATE)
fw_samp = fw_samp.set_index(fw_samp.DATE)
fw_samp.head()

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_PRCP,DES_SNDP,DES_FRSHTT
DATE,,,,,,,,,,,,,,,,,,,,,
2015-03-28,6,EV,2509,N902EV,DFW,GGG,1010,1005.0,-5.0,19.0,...,24.0,10.0,24.0,5.0,24.0,11.1,19.0,0.00,0.0,0.0
2015-03-28,6,MQ,3392,N682MQ,CLE,ORD,1010,1028.0,18.0,11.0,...,24.0,10.0,24.0,5.9,24.0,12.0,0.0,0.01,1.2,0.0
2015-03-28,6,US,621,N652AW,PHX,SNA,1020,1017.0,-3.0,28.0,...,24.0,8.6,24.0,4.7,24.0,9.9,0.0,0.00,0.0,0.0
2015-03-28,6,US,623,N195UW,PHX,SAN,1020,1014.0,-6.0,36.0,...,24.0,6.2,24.0,4.3,24.0,9.9,0.0,0.00,0.0,0.0
2015-03-28,6,WN,2455,N387SW,STL,TPA,1020,1023.0,3.0,9.0,...,24.0,10.0,24.0,8.8,24.0,22.0,0.0,0.27,0.0,0.0


Let's group by month and then compute some descriptive analytics about each month.

In [74]:
g = fw_samp.groupby(pd.TimeGrouper(freq='M'))

stats = g.describe()

Since the dataset has a little over 70 variables, let's just take a look
as two of them for the time being.

In [76]:
stats[['DEPARTURE_DELAY', 'OR_PRCP']]

DEPARTURE_DELAY                                              \
                     count       mean        std   min  25%  50%   75%   
DATE                                                                     
2015-03-31          2018.0   4.333994  25.475205 -22.0 -5.0 -3.0   3.0   
2015-04-30         15971.0   7.269551  31.755706 -26.0 -5.0 -2.0   5.0   
2015-05-31         16280.0   9.373833  36.538993 -39.0 -5.0 -2.0   7.0   
2015-06-30         16397.0  13.893212  43.732134 -33.0 -4.0 -1.0  12.0   
2015-07-31         17361.0  11.369334  36.835792 -26.0 -4.0 -1.0  10.0   
2015-08-31         16968.0   9.406000  33.980351 -35.0 -5.0 -1.0   8.0   
2015-09-30         15450.0   4.696375  29.756120 -25.0 -5.0 -3.0   2.0   
2015-10-31         16012.0   5.360042  33.778069 -36.0 -5.0 -2.0   3.0   
2015-11-30         15499.0   6.995355  35.432896 -32.0 -5.0 -2.0   4.0   
2015-12-31         15670.0  11.611806  42.188184 -35.0 -4.0 -1.0  10.0   

                    OR_PRCP                                                  
               max    count      mean       std  min  25%  50%   75%    max  
DATE                                                                         
2015-03-31   419.0   2028.0  0.132939  3.140399  0.0  0.0  0.0  0.01  99.99  
2015-04-30   698.0  16124.0  0.211296  3.060462  0.0  0.0  0.0  0.07  99.99  
2015-05-31   791.0  16474.0  0.214707  2.836771  0.0  0.0  0.0  0.04  99.99  
2015-06-30   902.0  16689.0  0.297219  3.882401  0.0  0.0  0.0  0.07  99.99  
2015-07-31   762.0  17511.0  0.310232  4.657871  0.0  0.0  0.0  0.02  99.99  
2015-08-31   729.0  17128.0  0.238954  3.750735  0.0  0.0  0.0  0.00  99.99  
2015-09-30   848.0  15516.0  0.173859  2.560629  0.0  0.0  0.0  0.01  99.99  
2015-10-31  1022.0      0.0       NaN       NaN  NaN  NaN  NaN   NaN    NaN  
2015-11-30   845.0  15660.0  0.250121  3.406302  0.0  0.0  0.0  0.04  99.99  
2015-12-31  1239.0  15928.0  0.239628  2.989402  0.0  0.0  0.0  0.08  99.99

Let's now compute the divergence of median/mean.

$$Median_n - Mean_n$$

In [82]:
# first we need to get the median of each variable.
# we have to keep in mind here that the median is going to
# be the median of the data sorted by the date index, not the median
# of the data sorted numerically.


In [77]:
stats

DAY_OF_WEEK                                               \
                 count      mean       std  min  25%  50%  75%  max   
DATE                                                                  
2015-03-31      2028.0  3.700690  2.591714  1.0  1.0  2.0  7.0  7.0   
2015-04-30     16124.0  3.906909  1.950406  1.0  2.0  4.0  6.0  7.0   
2015-05-31     16474.0  4.086439  1.997736  1.0  2.0  4.0  6.0  7.0   
2015-06-30     16689.0  3.750255  2.015714  1.0  2.0  4.0  5.0  7.0   
2015-07-31     17511.0  3.920621  1.901781  1.0  2.0  4.0  5.0  7.0   
2015-08-31     17128.0  3.999591  2.084049  1.0  2.0  4.0  6.0  7.0   
2015-09-30     15516.0  3.819992  1.952980  1.0  2.0  4.0  5.0  7.0   
2015-10-31     16108.0  4.010119  1.931389  1.0  2.0  4.0  6.0  7.0   
2015-11-30     15660.0  3.942529  2.100701  1.0  2.0  4.0  6.0  7.0   
2015-12-31     15928.0  3.862757  1.954913  1.0  2.0  4.0  5.0  7.0   

           FLIGHT_NUMBER                 ...    DES_SNDP       DES_FRSHTT  \
                   count         mean    ...         75%   max      count   
DATE                                     ...                                
2015-03-31        2028.0  2179.758383    ...         0.0  14.2     2028.0   
2015-04-30       16124.0  2205.330501    ...         0.0  13.0    16124.0   
2015-05-31       16474.0  2147.820626    ...         0.0   7.1    16474.0   
2015-06-30       16689.0  2163.208640    ...         0.0   0.0    16689.0   
2015-07-31       17511.0  2170.479527    ...         0.0   0.0    17511.0   
2015-08-31       17128.0  2162.005722    ...         0.0   0.0    17128.0   
2015-09-30       15516.0  2142.127417    ...         0.0   5.1    15516.0   
2015-10-31       16108.0  2108.873107    ...         NaN   NaN        0.0   
2015-11-30       15660.0  2154.298020    ...         0.0  15.0    15660.0   
2015-12-31       15928.0  2142.371735    ...         0.0  22.8    15928.0   

                                                                          
                    mean           std  min  25%  50%      75%       max  
DATE                                                                      
2015-03-31   8188.111440  22725.786871  0.0  0.0  0.0  10000.0  111000.0  
2015-04-30  10049.744294  24703.979729  0.0  0.0  0.0  10000.0  111110.0  
2015-05-31  10517.825907  25170.358102  0.0  0.0  0.0  10000.0  111110.0  
2015-06-30   9488.178621  23466.539464  0.0  0.0  0.0  10010.0  110110.0  
2015-07-31   6643.038719  18336.574733  0.0  0.0  0.0  10000.0  110010.0  
2015-08-31   6409.386794  19039.655660  0.0  0.0  0.0  10000.0  110110.0  
2015-09-30   7914.441415  22554.797168  0.0  0.0  0.0  10000.0  111000.0  
2015-10-31           NaN           NaN  NaN  NaN  NaN      NaN       NaN  
2015-11-30  12731.782312  29499.584120  0.0  0.0  0.0  10000.0  111010.0  
2015-12-31  18952.034154  36509.233635  0.0  0.0  0.0  10000.0  111010.0  

[10 rows x 456 columns]